### EDA : 시계열 그래프

#### * 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

In [ ]:
# 그래프 기본설정
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'Malgun Gothic'

#### * 파일 불러오기

In [ ]:
final=pd.read_excel('최종 데이터셋.xlsx')

#### * 연도별로 나누기

In [ ]:
data21 = final[final.Year == 2021]
data20 = final[final.Year == 2020]
data19 = final[final.Year == 2019]
data18 = final[final.Year == 2018]
data17 = final[final.Year == 2017]
data16 = final[final.Year == 2016]

#### 1. 연도별 리뷰 데이터의 개수 분포 파악

* 'Year'열을 기준으로 빈도수를 count한 피벗 테이블 생성

In [ ]:
piv1 = pd.pivot_table(data = final,
              index = ['Year'],
              aggfunc = 'count')

* 그래프 시각화

In [ ]:
## 데이터 연도별 빈도
month_axis = np.arange(len(piv1.index.tolist()))
month_axis[:] = piv1.comment.mean()
sns.catplot(data = piv1,kind = 'bar',x = piv1.index,  y = 'comment', alpha=.6, height=6)
plt.plot(np.arange(len(piv1.index.tolist())),month_axis, c = 'r', linestyle = '--')
plt.text(len(piv1.index.tolist())-1,piv1.comment.mean(),"Mean: %.f" % piv1.comment.mean(), fontsize = 18)
plt.title("Year")
plt.show()

#### 2. 전체 명사 상위 100개 워드 클라우드

* Counter를 이용한 명사 단순 빈도 분석

In [ ]:
#### 명사 단순빈도분석

from collections import Counter

#코퍼스 생성
corpus = []

final['nouns'] = [text.split(" ") for text in final['nouns']]

for i in range(len(final['nouns'])):
    corpus.append(final['nouns'][i])

In [ ]:
lst = []
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        lst.append(corpus[i][j])
        
count = Counter(lst)
most_count = count.most_common(100) #상위 100개 단어 확인

* 한글 글꼴 깨짐 방지

In [ ]:
#한글 글꼴 깨짐 방지
import matplotlib
from matplotlib import font_manager, rc
import platform

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

* 워드클라우드 시각화

In [ ]:
## 워드클라우드
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8,8))
fig, ax = plt.subplots(1,1)
wordcloud=WordCloud(font_path=r'C:\Users\user\Desktop\대학생활\글씨체\NanumSquare_acEB.ttf', background_color='white').generate(str(most_count))
ax.set_title('전체 명사 상위 100')
plt.axis('off')
ax.imshow(wordcloud)
plt.savefig('전체 명사 상위 100.png')

#### 3. 특정 키워드 언급량 추이 시각화

* 필요한 함수 정의

In [ ]:
# 해당연도의 데이터를 반환하는 함수
def get_com(year):  
    return data[data.Year == year].reset_index(drop=True)

In [ ]:
def dailycount_comment(start_day, end_day, keyword):
    """
    start_day 부터 end_day까지의 일간 게시글 중에서 
    keyword가 포함된 게시글 수를 추출하여 데이터프레임으로 반환
    """
    
    # 시간 기준 적용
    selected = data[(data.date >= start_day) & (data.date <= end_day)].sort_values('date').reset_index(drop=True)

    
    result = [] # 일별 게시글 수를 저장할 리스트
    
    for day in pd.date_range(start=start_day, end=end_day, freq="D"):
        count = 0
        
        for text in selected[selected.date == day]['comment']:
            if re.search(keyword, text): # 댓글에서 keyword를 찾아서 있다면 count 증가
                count += 1

        result.append(count) # 날짜 순으로 게시글 수를 추가함
        
    return result

* 전기차 관련 키워드 언급량 추이 

In [ ]:
# 2019년부터 전기차와 관련한 키워드를 가진 댓글 수 그래프
start = "2019-01-01"
end = "2021-07-31" 

keyword_list = ['전기차', '아이오닉', '충전', '테슬라']

values = np.array([dailycount_comment(start, end, keyword) for keyword in keyword_list]).T
dates = pd.date_range(start, end, freq="D")
df = pd.DataFrame(values, dates, columns=keyword_list)
df = df.rolling(30).mean() #30일 이동평균

plt.figure(figsize=(12,5))
g = sns.lineplot(data=df, palette="tab10", linewidth=2.5)
g.set_title('전기차 관련 키워드 언급량')

* 전기차 모델 별 언급량 추이

In [ ]:
# 2019년부터 전기차 모델별 댓글 수 그래프
start = "2019-01-01"
end = "2021-07-31" 

keyword_list = ['모델 Y', '모델 3', '아이오닉', '코나', '니로', 'EQA']

values = np.array([dailycount_comment(start, end, keyword) for keyword in keyword_list]).T
dates = pd.date_range(start, end, freq="D")
df = pd.DataFrame(values, dates, columns=keyword_list)
df = df.rolling(30).mean() #30일 이동평균

plt.figure(figsize=(12,5))
g = sns.lineplot(data=df, palette="tab10", linewidth=2)
g.set_title('전기차 모델별 언급량')